# Bibliotecas

In [1]:
import pandas as pd
from pathlib import Path
import os

# Tratar bases

In [2]:
# Diretório base onde o arquivo CSV está localizado
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base'

# Definir o caminho completo da base não tratada do ibovespa
arquivo_ibovespa = caminho / 'Dados Históricos - Ibovespa.csv'

acao = pd.read_csv(arquivo_ibovespa)
acao

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,05.01.2015,47.517,48.512,48.512,47.264,"3,87M","-2,05%"
1,06.01.2015,48.001,47.517,48.061,47.338,"4,56M","1,02%"
2,07.01.2015,49.463,48.006,49.882,48.006,"4,41M","3,05%"
3,08.01.2015,49.943,49.463,50.261,49.017,"3,62M","0,97%"
4,09.01.2015,48.840,49.955,49.955,48.501,"3,00M","-2,21%"
...,...,...,...,...,...,...,...
2609,14.07.2025,135.299,136.187,136.187,134.840,"7,33B","-0,65%"
2610,15.07.2025,135.250,135.298,136.022,134.380,"6,90B","-0,04%"
2611,16.07.2025,135.511,135.250,135.641,134.265,"7,83B","0,19%"
2612,17.07.2025,135.565,135.515,135.792,135.016,"6,80B","0,04%"


In [3]:
def tratar_dados(caminho_csv):
    # Força leitura como string para evitar perda de zeros
    acao = pd.read_csv(caminho_csv, dtype=str)

    # Converter a coluna de data
    acao['Data'] = pd.to_datetime(acao['Data'], format='%d.%m.%Y')

    # Corrigir colunas numéricas
    colunas_dinheiro = ['Último', 'Abertura', 'Máxima', 'Mínima']
    for coluna in colunas_dinheiro:
        acao[coluna] = (
            acao[coluna]
            .str.replace('.', '', regex=False)  # Remove separador de milhar
            .str.replace(',', '', regex=False)  # Remove separador decimal
            .astype(float)
        )

    # Corrigir variação percentual
    acao['Var%'] = (
        acao['Var%']
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )
    acao['Var%'] = acao['Var%'].round(4)
    acao = acao.rename(columns={'Var%': 'Variacao (%)'})

    # Remover a coluna de volume
    acao = acao.drop(columns=['Vol.'])

    # Renomeando coluna do valor de fechamento do dia 
    acao = acao.rename(columns={'Último': 'Fechamento'})

    # Ordenar por data
    acao = acao.sort_values(by='Data')

    return acao

# Aplicar função e salvar resultado
df_ibovespa = tratar_dados(arquivo_ibovespa)
df_ibovespa.to_csv("Base Ibovespa Tratada.csv", index=False)


In [4]:
df_ibovespa

,Data,Fechamento,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47517.0,48512.0,48512.0,47264.0,-0.0205
1,2015-01-06,48001.0,47517.0,48061.0,47338.0,0.0102
2,2015-01-07,49463.0,48006.0,49882.0,48006.0,0.0305
3,2015-01-08,49943.0,49463.0,50261.0,49017.0,0.0097
4,2015-01-09,48840.0,49955.0,49955.0,48501.0,-0.0221
...,...,...,...,...,...,...
2609,2025-07-14,135299.0,136187.0,136187.0,134840.0,-0.0065
2610,2025-07-15,135250.0,135298.0,136022.0,134380.0,-0.0004
2611,2025-07-16,135511.0,135250.0,135641.0,134265.0,0.0019
2612,2025-07-17,135565.0,135515.0,135792.0,135016.0,0.0004


In [5]:
df_ibovespa.dtypes

Data            datetime64[ns]
Fechamento             float64
Abertura               float64
Máxima                 float64
Mínima                 float64
Variacao (%)           float64
dtype: object